<a href="https://colab.research.google.com/github/tunjis/Data-Analysis-Visualisation_Excel/blob/main/bitcoin_halving_supply_visualiser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
# Plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Date/Time
from datetime import datetime, timedelta
from dateutil import tz
# Data Handling & Calculation
from tabulate import tabulate
import math
import sys

# --- Constants ---
START_DATE = datetime(2009, 1, 3, 0, 0, 0, tzinfo=tz.UTC) # Genesis block time (approx)
BLOCK_TIME_MINUTES = 10  # Average block time target
BLOCKS_PER_HALVING = 210000
INITIAL_REWARD = 50.0
MAX_BITCOIN = 21_000_000
SATOSHI = 1e-8 # Smallest Bitcoin unit

def calculate_bitcoin_issuance_schedule():
    """
    Calculates the full halving schedule including dates, rewards,
    and cumulative BTC. Returns data structured for analysis and plotting.
    (No changes needed in this function)
    """
    schedule_data = []
    halving_dates = []
    cumulative_btc_points = []
    reward_points = []

    current_block = 0
    total_btc = 0.0
    halving_index = 0
    current_time = START_DATE

    while True:
        reward = INITIAL_REWARD / (2 ** halving_index)
        reward = max(reward, 0)

        if reward * BLOCKS_PER_HALVING < SATOSHI and halving_index > 0:
            break

        start_block_this_period = current_block
        theoretical_max_blocks = start_block_this_period + BLOCKS_PER_HALVING
        btc_remaining_to_issue = MAX_BITCOIN - total_btc

        if reward < SATOSHI:
             blocks_to_issue_this_period = 0
             btc_this_period = 0
        else:
             blocks_can_be_issued = math.floor(btc_remaining_to_issue / reward) if reward > 0 else 0
             blocks_this_period = min(BLOCKS_PER_HALVING, blocks_can_be_issued)
             btc_this_period = blocks_this_period * reward
             if total_btc + btc_this_period > MAX_BITCOIN:
                 btc_this_period = MAX_BITCOIN - total_btc
                 blocks_this_period = int(btc_this_period / reward) if reward > 0 else 0
                 btc_this_period = max(0, btc_this_period)

        if blocks_this_period <= 0 and total_btc >= MAX_BITCOIN - SATOSHI:
            break

        end_block_this_period = start_block_this_period + blocks_this_period - 1
        start_time_this_period = current_time
        duration_this_period = timedelta(minutes=blocks_this_period * BLOCK_TIME_MINUTES)
        end_time_this_period = start_time_this_period + duration_this_period

        # Use start date for points, consistent with Matplotlib version
        halving_dates.append(start_time_this_period)
        cumulative_btc_points.append(total_btc) # Cumulative *before* this period for plotting start point
        reward_points.append(reward) # Reward *during* this period

        schedule_data.append({
            "halving_index": halving_index,
            "start_block": start_block_this_period,
            "end_block": end_block_this_period,
            "start_date": start_time_this_period,
            "end_date": end_time_this_period,
            "reward": reward,
            "btc_issued": btc_this_period,
            "cumulative_btc": total_btc + btc_this_period
        })

        total_btc += btc_this_period
        current_block += blocks_this_period
        current_time = end_time_this_period
        halving_index += 1

        if halving_index > 64:
            print("Warning: Exceeded 64 halving epochs.", file=sys.stderr)
            break

    # Add final points for plotting to show the plateau/zero reward state
    if schedule_data:
        last_period_end_time = schedule_data[-1]['end_date']
        final_btc_total = schedule_data[-1]['cumulative_btc']
        plot_end_time = last_period_end_time + timedelta(days=365*5) # Extend plot lines slightly
        # Add point for final level
        halving_dates.append(last_period_end_time)
        cumulative_btc_points.append(final_btc_total)
        reward_points.append(0) # Reward becomes 0
        # Add one more point far in future to draw the flat line
        halving_dates.append(plot_end_time)
        cumulative_btc_points.append(final_btc_total)
        reward_points.append(0)


    return schedule_data, halving_dates, cumulative_btc_points, reward_points

# --- NEW Plotting function using Plotly ---
def generate_plotly_chart(halving_dates, cumulative_btc_points, reward_points, current_time_utc):
    """Generates an interactive plot using Plotly."""
    if not halving_dates or not cumulative_btc_points or not reward_points:
        print("Error: Not enough data to generate plot.", file=sys.stderr)
        return

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # --- Add Cumulative BTC Supply trace (Left Y-axis) ---
    fig.add_trace(
        go.Scatter(
            x=halving_dates,
            y=[btc / 1_000_000 for btc in cumulative_btc_points], # Convert to millions
            name="Cumulative Supply (Millions)",
            line=dict(color='royalblue', width=2), # Equivalent to tab:blue
            hovertemplate = "Date: %{x|%Y-%m-%d}<br>Supply: %{y:.2f}M BTC<extra></extra>" # Custom hover text
        ),
        secondary_y=False, # Assign to the primary (left) y-axis
    )

    # --- Add Block Reward trace (Right Y-axis) ---
    # Use line_shape='hv' for step-like appearance (Horizontal-Vertical steps)
    fig.add_trace(
        go.Scatter(
            x=halving_dates,
            y=reward_points,
            name="Block Reward (BTC)",
            line=dict(color='firebrick', width=2, dash='dash', shape='hv'), # Equivalent to tab:red, dashed line
            hovertemplate = "Date: %{x|%Y-%m-%d}<br>Reward: %{y:.8f} BTC<extra></extra>" # Custom hover text
        ),
        secondary_y=True, # Assign to the secondary (right) y-axis
    )

    # --- Add Vertical Line for Current Time ---
    fig.add_vline(
        x=current_time_utc.timestamp() * 1000, # Plotly uses milliseconds since epoch for vline
        line_width=1.5,
        line_dash="dot",
        line_color="black",
        annotation_text=f" Current ({current_time_utc.strftime('%Y-%m-%d')})", # Optional text near line
        annotation_position="top right"
    )

    # --- Add Dummy Trace for the Legend Entry --- <<<< NEW PART ADDED HERE
    # This trace has no visible points but provides the legend item
    fig.add_trace(go.Scatter(
        x=[None], # No data points
        y=[None],
        mode='lines', # Need 'lines' mode to show line style in legend
        name=f"Current ({current_time_utc.strftime('%Y-%m-%d')})", # The label for the legend
        line=dict(color='black', dash='dot', width=1.5), # Match the vline style
        showlegend=True # Make sure it appears in the legend
    ))

    # --- Update layout ---
    fig.update_layout(
    title_text="Bitcoin Issuance: Cumulative Supply & Block Reward Halvings",
    xaxis_title="Year",

     # Configure the primary (Left) Y-axis (Cumulative Supply - Blue)
    yaxis=dict(
        title=dict(
            text="Cumulative Bitcoin Supply (Millions)",
            font=dict(
                color="royalblue"  # Title color (already set)
            )
        ),
        range=[0, MAX_BITCOIN / 1_000_000 * 1.05], # Range (already set)
        tickfont=dict(
            color="royalblue"  # <<< Color for the tick labels (0, 5, 10...)
        ),
        linecolor="royalblue"   # <<< Color for the axis line itself
    ),

    # Configure the secondary (Right) Y-axis (Block Reward - Red)
    yaxis2=dict(
        title=dict(
            text="Block Reward (BTC)",
            font=dict(
                color="firebrick" # Title color (already set)
            )
        ),
        range=[0, INITIAL_REWARD * 1.1], # Range (already set)
        tickfont=dict(
            color="firebrick"  # <<< Color for the tick labels (0, 10, 20...)
        ),
        linecolor="firebrick"   # <<< Color for the axis line itself
    ),
        legend_title_text="Legend",
        # Position legend top left
        legend=dict(
            yanchor="top",
            y=0.98,
            xanchor="right",
            x=0.98,
            bgcolor='rgba(255,255,255,0.7)' # Semi-transparent background
        ),
        hovermode="x unified" # Show hover info for both lines at same x-position
    )

    # Set y-axis ranges (optional, but good for consistency)
    fig.update_yaxes(title_text="Cumulative Bitcoin Supply (Millions)", secondary_y=False, range=[0, MAX_BITCOIN / 1_000_000 * 1.05])
    fig.update_yaxes(title_text="Block Reward (BTC)", secondary_y=True, range=[0, INITIAL_REWARD * 1.1])

    # Set x-axis range
    plot_start_date = START_DATE - timedelta(days=30)
    plot_end_date = datetime(2145, 1, 1, tzinfo=tz.UTC)
    if halving_dates:
        plot_end_date = max(plot_end_date, halving_dates[-1])
    fig.update_xaxes(range=[plot_start_date, plot_end_date])


    # --- Show the figure ---
    # This will usually open the chart in your default web browser
    fig.show()


# --- Main Execution ---
if __name__ == "__main__":
    # --- Time Setup ---
    utc_now = datetime.now(tz.UTC)

    # --- Calculate Full Schedule & Data ---
    schedule_data, plot_dates, plot_cumulative_btc, plot_rewards = calculate_bitcoin_issuance_schedule()

    # --- Calculate Current Progress ---
    minutes_elapsed = (utc_now - START_DATE).total_seconds() / 60
    current_total_blocks_estimated = max(0, int(minutes_elapsed // BLOCK_TIME_MINUTES))

    current_total_btc_mined = 0
    current_period_index = -1
    current_reward = 0
    temp_blocks_count = 0

    if schedule_data:
        for i, period in enumerate(schedule_data):
            blocks_in_period = period['end_block'] - period['start_block'] + 1
            if temp_blocks_count + blocks_in_period >= current_total_blocks_estimated:
                current_period_index = i
                current_reward = period['reward']
                blocks_mined_this_period = current_total_blocks_estimated - temp_blocks_count
                current_total_btc_mined += blocks_mined_this_period * period['reward']
                break
            else:
                current_total_btc_mined += period['btc_issued']
                temp_blocks_count += blocks_in_period
        if current_period_index == -1:
             current_period_index = len(schedule_data) -1
             current_total_btc_mined = MAX_BITCOIN
             current_reward = 0

    current_total_btc_mined = min(current_total_btc_mined, MAX_BITCOIN)

    # --- Estimated Time Remaining ---
    final_block_time_estimate = datetime(2140, 1, 1, tzinfo=tz.UTC)
    if schedule_data:
        for period in reversed(schedule_data):
            if period['btc_issued'] > SATOSHI:
                final_block_time_estimate = period['end_date']
                break

    time_remaining = final_block_time_estimate - utc_now

    # --- Presentation (Text Output) ---
    print("--- Bitcoin Issuance Status ---")
    print(f"🕒 Calculation Time (UTC): {utc_now.strftime('%Y-%m-%d %H:%M:%S')} UTC")

    print(f"\n🧮 Bitcoin Supply Formula Sum:")
    print(r"   Total BTC = Σ [ Reward(n) * BlocksPerPeriod ] ≈ 21,000,000 BTC")
    print(r"   where Reward(n) = 50 / (2^n) BTC for halving epoch n")

    print(f"\n📊 Current Progress (Estimated based on time):")
    print(f"   • Total Blocks Mined : {current_total_blocks_estimated:,}")
    print(f"   • Current Block Reward: {current_reward:.8f} BTC")
    print(f"   • Total Bitcoin Mined: {current_total_btc_mined:,.2f} BTC")
    print(f"   • Percentage of Max Supply Mined: {(current_total_btc_mined / MAX_BITCOIN * 100):.2f}%")

    # --- Countdown ---
    print(f"\n⏳ Estimated Time Until Last Significant BTC Emission (~{final_block_time_estimate.year}):")
    if time_remaining.total_seconds() > 0:
        years, remainder = divmod(time_remaining.total_seconds(), 365.25 * 24 * 3600)
        days, remainder = divmod(remainder, 24 * 3600)
        hours, remainder = divmod(remainder, 3600)
        minutes, _ = divmod(remainder, 60)
        print(f"   {int(years)} years, {int(days)} days, {int(hours)} hours, {int(minutes)} minutes remaining")
    else:
         print(f"   The estimated time (~{final_block_time_estimate.strftime('%Y-%m-%d')}) has passed.")

    # --- Table ---
    print("\n📜 Bitcoin Issuance Schedule (Estimated Dates):")
    table_output_data = []
    headers = ["", "Halving #", "Start Block", "End Block", "~Start Date", "Reward (BTC)", "BTC Issued", "Cumulative BTC"]

    if not schedule_data:
         print("   No schedule data generated.")
    else:
        added_ellipsis = False
        for i, period in enumerate(schedule_data):
            marker = "->" if i == current_period_index else ""

            if i <= 10 or i == current_period_index :
                 table_output_data.append([
                    marker,
                    period['halving_index'],
                    f"{period['start_block']:,}",
                    f"{period['end_block']:,}",
                    f"{period['start_date'].strftime('%Y-%m-%d')}",
                    f"{period['reward']:.8f}",
                    f"{period['btc_issued']:,.2f}",
                    f"{period['cumulative_btc']:,.2f}"
                ])
            elif not added_ellipsis and i > 10:
                 table_output_data.append(['...', '...', '...', '...', '...', '...', '...', '...'])
                 added_ellipsis = True

        print(tabulate(table_output_data, headers=headers, tablefmt="github", numalign="right", stralign="right"))

    # --- Generate Interactive Plot using Plotly ---
    print("\n📈 Generating Interactive Plot (using Plotly)...")
    generate_plotly_chart(plot_dates, plot_cumulative_btc, plot_rewards, utc_now)

--- Bitcoin Issuance Status ---
🕒 Calculation Time (UTC): 2025-04-25 01:15:48 UTC

🧮 Bitcoin Supply Formula Sum:
   Total BTC = Σ [ Reward(n) * BlocksPerPeriod ] ≈ 21,000,000 BTC
   where Reward(n) = 50 / (2^n) BTC for halving epoch n

📊 Current Progress (Estimated based on time):
   • Total Blocks Mined : 857,671
   • Current Block Reward: 3.12500000 BTC
   • Total Bitcoin Mined: 19,742,721.88 BTC
   • Percentage of Max Supply Mined: 94.01%

⏳ Estimated Time Until Last Significant BTC Emission (~2140):
   115 years, 165 days, 4 hours, 44 minutes remaining

📜 Bitcoin Issuance Schedule (Estimated Dates):
|     |   Halving # |   Start Block |   End Block |   ~Start Date |   Reward (BTC) |    BTC Issued |   Cumulative BTC |
|-----|-------------|---------------|-------------|---------------|----------------|---------------|------------------|
|     |           0 |             0 |     209,999 |    2009-01-03 |    50.00000000 | 10,500,000.00 |    10,500,000.00 |
|     |           1 |       2